In [1]:
import json
import pandas as pd
df = pd.read_json('champions.json')
Dict_id_to_name=pd.Series(df.Name.values,index=df.ChampId).to_dict()
Dict_name_to_id=pd.Series(df.ChampId.values,index=df.Name).to_dict()

In [2]:
Dict_name_to_id['Teemo']

17

In [3]:
#Reconocer los campeones

#Input:
#(Otra idea es hacer un array de 5 en el orden exacto)
#[Singed,None,Zed,None,None] evtas problemas doble linea
#Aliados {[Voli,JUNG],[Lulu,SUPP]}
#Enemigos {[Singed,TOP],[Zed,MID]}
#Nosotros queremos un TOP

rol_to_array={'Top':0,'Jng':1,'Mid':2,'Adc':3,'Sup':4}

ally_team=[None,106,None,None,117]
enemy_team=[27,None,238,None,None]
Rol_selected='Top'


class Champ:
    def __init__(self, name, Id,Time,Rol,Linea,Damages,Caracteristicas,Facilidad):
        self.Nombre=name
        self.Id=Id
        self.Time=Time #No tengo ni idea de como funciona
        self.Rol=Rol#['Luchador','Tank']
        self.Linea=Linea#['Top',None]
        self.Damages=Damages#{'AD':0,'AP':0,'TD':0}
        self.Caracteristicas=Caracteristicas#{'Engage':0,'Disengage':0}
        self.Facilidad=Facilidad#0 #0,1,2?
        #Vamos a divivir los damages y el engage y disengage
        WrAgainstlane=0
        WrAgainstGlobal=0
        WrWith=0
        DmgAport={}
        EquilibrioDMG=0
        
        
def Calc_lanewr(champ_class):
    if champ_class.Id==75:
        return 49.64
    if champ_class.Id==150:
        return 48.2
    if champ_class.Id==17:
        return 47.1
    
def get_wr(ally_id,enemy_id):
    if enemy_id==27:
        return 49.64
    if enemy_id==238:
        return 53
    
def get_wrwith(ally_id_champ,ally_id):
    if ally_id==27:
        return 49.64
    if ally_id==238:
        return 53
    return 50

        
def Calc_againstwr(champ_class):
    sumawr=0
    divisor=0
    for enemy in enemy_team:
        if enemy!=None:
            #query para coger el wr against para los campeones de Enemigos
            wr_temp=get_wr(champ_class,enemy)
            sumawr+=wr_temp
            divisor+=1
    return sumawr/divisor

def Calc_compawr(champ_class,Rol,multiplicador=1):
    suma=0
    divisor=0
    for ally in ally_team:
        if ally != None:
            if Rol in ['Sup','Adc'] and x.Rol in ['Sup','adc']:
                suma+=get_wrwith(champ_class,ally)*multiplicador
                divisor+=1*multiplicador
            else:
                suma+=get_wrwith(champ_class,ally)
                divisor+=1
    return suma/divisor
        


def dmg_calc(champ_class):#podriamos dar proporciones al impacto segun el rol
    dmg={'AD':0,'AP':0,'TD':0}
    num=1
    for ally in ally_team:
        if ally != None:
            num+=1
            #ponemos los damages
            dmg['AD']+=40
            dmg['AP']+=50
            dmg['TD']+=10
            
    dmg['AD']=(champ_class.Damages['AD']+dmg['AD'])/num
    dmg['AP']=(champ_class.Damages['AP']+dmg['AP'])/num
    dmg['TD']=(champ_class.Damages['TD']+dmg['TD'])/num
    
    return dmg

        
Nasus_Champ=Champ('Nasus',75,'001122',['Luchador','Tank'],['Top','Mid'],{'AD':70,'AP':30,'TD':0},{'Engage':0,'Disengage':0},1)
Gnar_Champ=Champ('Gnar',150,'021221',['Luchador','Tank'],['Top','Mid'],{'AD':80,'AP':20,'TD':0},{'Engage':1,'Disengage':0},1)
Teemo_Champ=Champ('Teemo',17,'021021',['Mago','None'],['Top','Mid'],{'AD':20,'AP':80,'TD':0},{'Engage':0,'Disengage':1},1)
   
#Queremos generar una recomendacion
#Recomm=[['Puntuation','Champ1']]
Rec=[]

#Cogemos a los campeones que puedan ser toplaner, (Aquellos que primera o segunda linea son tops)
Lista_playable=[Nasus_Champ,Gnar_Champ,Teemo_Champ]
Lista_posicion=[]

for campeon in Lista_playable:
    
    if Rol_selected in campeon.Linea:
        Lista_posicion.append(campeon)

print(Lista_posicion)
for playable in Lista_posicion:
        playable.WrAgainstlane=Calc_lanewr(playable)
        playable.WrAgainstGlobal=Calc_againstwr(playable)
        playable.WrWith=Calc_compawr(playable,Rol_selected)
        playable.DmgAport=dmg_calc(playable)
        playable.EquilibrioDMG=min(playable.DmgAport['AD']/playable.DmgAport['AP'],playable.DmgAport['AP']/playable.DmgAport['AD'])+1
        


#Multiplicadores
TeamNeedsATank=0.1 #Miramos si hay un tanque o fighter en el equipo aliado


#Aqui abajo no tenemos en cuenta si es supp o adc, el equilibrio de daño es muy grande ***** 
HasEquilibrateDmg=0.1 #Proporcional al equilibrio, optimo 50/50


TimeStamp=0 #0 early, 0.05 mid, 0.1 late #Ya que los enemigos no cierran partidas(Maybe multiplicar por el numero si no cmabiamos el sistema)
Facilidad=0.1 #campeon facil 0.1, campeon dificil 0
TeamHasEngageANDDisengage=0.1 #0.1 si el equipo tiene las dos, 0.05 si solo tiene una, 0.0 si ninguna


#calculo_final
#multiplicadores=sum(multiplicadores...)
#calculo=(wrAgainstLane*m1+wrAgainstGlobal*m2+wrWith*m3)*(multiplicadores*correcion)
print('###########################')
for playable in Lista_posicion:
    print(playable.Nombre)
    print(playable.WrAgainstlane)
    print(playable.WrAgainstGlobal)
    print(playable.WrWith)
    print(playable.EquilibrioDMG)
    print('###########Calculo###########')
    calc=(playable.WrAgainstlane+playable.WrAgainstGlobal+playable.WrWith)/3 +playable.EquilibrioDMG*0.1
    print(calc)
    print('#############################')

[<__main__.Champ object at 0x000001B31E3525B0>, <__main__.Champ object at 0x000001B31E352610>, <__main__.Champ object at 0x000001B31E352700>]
###########################
Nasus
49.64
51.32
50.0
1.8666666666666667
###########Calculo###########
50.50666666666667
#############################
Gnar
48.2
51.32
50.0
1.75
###########Calculo###########
50.015
#############################
Teemo
47.1
51.32
50.0
1.5555555555555556
###########Calculo###########
49.628888888888895
#############################
